In [23]:
import h5py
import matplotlib.pyplot as plt

In [ ]:
print("hello world")

### Read the provided h5 file from Eric and shorten it to have 1000 data points from index 7000 - 8000 of the original data file

In [33]:

with h5py.File("./tests/data/points3d_transformed.h5", 'r') as f:
    # print(f.keys())
    dset = f["tracks"]
    print(f'shape of original file {dset.shape}')

    shorten_file = h5py.File('./tests/data/points3d_transformed_shorten.h5', 'w')
    shorten_file.create_dataset('tracks', data=dset[7000:8000,])
    shorten_file.close()

shape of original file (123548, 1, 3, 3)


In [39]:
# sanity check for the new data file

with h5py.File("./tests/data/points3d_transformed_shorten.h5", 'r') as f:
    print(f'shape of shortened file {f["tracks"].shape}')

shape of shortened file (1000, 1, 3, 3)


### Run Stac MJX

In [1]:
import stac_mjx 
from pathlib import Path

# Enable XLA flags if on GPU
stac_mjx.enable_xla_flags()

# Choose parent directory as base path for data files
base_path = Path("/root/vast/joshua/stac-mjx")

# Load configs
cfg = stac_mjx.load_configs(base_path / "configs")
# cfg = stac_mjx.load_configs(base_path + "/configs")

# Load data
kp_data, sorted_kp_names = stac_mjx.load_data(cfg, base_path)

# Run stac
fit_path, ik_only_path = stac_mjx.run_stac(
 cfg,
 kp_data, 
 sorted_kp_names, 
 base_path=base_path
)

/root/vast/joshua/stac-mjx/stac_mjx/op_utils.py:16: DeprecationWarning: jax.lib.xla_bridge.get_backend is deprecated; use jax.extend.backend.get_backend.
  if xla_bridge.get_backend().platform == "gpu":


Calibration iteration: 1/6
Pose Optimization:
Pose Optimization done in 29.057384490966797
Frame 1 done in 29.012714385986328 with a final error of 0.0
Mean: 0.0
Standard deviation: 0.0
starting offset optimization
Begining offset optimization:


/root/miniforge3/envs/stac-mjx-env/lib/python3.11/site-packages/jaxopt/_src/optax_wrapper.py:120: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(update_fun, params, updates)


Final error of 0.000995560665614903
offset optimization finished in 9.746098279953003
Calibration iteration: 2/6
Pose Optimization:
Pose Optimization done in 0.45139145851135254
Frame 1 done in 0.44734621047973633 with a final error of 6.729706202024399e-12
Mean: 6.729706202024399e-12
Standard deviation: 0.0
starting offset optimization
Begining offset optimization:
Final error of 0.0009844849118962884
offset optimization finished in 7.126520395278931
Calibration iteration: 3/6
Pose Optimization:
Pose Optimization done in 0.024038076400756836
Frame 1 done in 0.019692182540893555 with a final error of 1.7186247217026995e-11
Mean: 1.7186247217026995e-11
Standard deviation: 0.0
starting offset optimization
Begining offset optimization:
Final error of 0.0009835025994107127
offset optimization finished in 0.06156659126281738
Calibration iteration: 4/6
Pose Optimization:
Pose Optimization done in 0.0176239013671875
Frame 1 done in 0.01408076286315918 with a final error of 1.5830088975765655e

In [3]:
import mujoco

In [4]:
mujoco.__version__

'3.2.2'